In [2]:
# Libraries

import websocket
import json
import pandas as pd
from binance.client import Client
from pandas.tseries.offsets import MonthEnd
from sqlalchemy import create_engine

import pymysql

# sleep function to avoid overloading API
from time import sleep

In [3]:
# Binance client initalization
client = Client()

In [4]:
# Defining sql Engine, our connection to sqlLite
# Engine to write and pull data with SQLLite

# functioning engine
# engine = create_engine('mysql+pymysql://root:')
engine = create_engine(R'sqlite:///C:\Users\hp\Desktop\Projects Coding\Crypto_Market_Analysis_Bot\\COINS_DATA.db', echo=True)


# testing engine
# engine = create_engine('sqlite:///Cointest.db')

In [5]:
# Function to fetch data for each coin from binance

def getdata(symbol, start):

    end = str(pd.to_datetime(start) + MonthEnd(0))
    frame = pd.DataFrame(client.get_historical_klines(symbol, '1m', start))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame.set_index('Time', inplace = True) 
    frame.index = pd.to_datetime(frame.index, unit = 'ms')
    frame = frame.astype(float)
    return frame

In [6]:
# Testing getdata function
df = getdata('BTCUSDT', 'December 1 2022')

In [14]:
df.tail(10)

,Open,High,Low,Close,Volume
Time,,,,,
2022-12-08 20:28:00,17220.25,17222.35,17216.93,17221.98,179.11093
2022-12-08 20:29:00,17221.98,17222.61,17208.96,17210.86,214.72006
2022-12-08 20:30:00,17210.85,17230.00,17209.79,17226.11,263.37162
2022-12-08 20:31:00,17225.45,17230.72,17222.17,17224.32,143.61547
2022-12-08 20:32:00,17224.32,17224.64,17209.30,17209.31,120.93537
2022-12-08 20:33:00,17209.85,17210.74,17199.92,17206.26,193.02434
2022-12-08 20:34:00,17206.25,17210.45,17201.47,17201.98,101.77985
2022-12-08 20:35:00,17202.67,17206.50,17198.31,17205.97,142.04144
2022-12-08 20:36:00,17205.97,17216.96,17204.71,17210.10,134.47562


In [7]:
coins = ('BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT', 'ADAUSDT', 
        'XRPUSDT' , 'DOTUSDT', 'LUNAUSDT', 'DOEGUSDT', 'AVAXUSDT',
        'SHIBUSDT', 'MATICUSDT', 'LTCUSDT', 'UNIUSDT', 'ALGOUSDT', 'TRXUSDT', 
        'LINKUSDT', 'MANAUSDT', 'ATOMUSDT', 'VETUSDT')

In [8]:
daterange = pd.date_range('September 01 2022', pd.to_datetime('today'), freq='MS')

In [11]:
engine.execute('CREATE DATABASE CoinIntra')

2022-12-08 15:42:03,665 INFO sqlalchemy.engine.Engine CREATE DATABASE CoinIntra
2022-12-08 15:42:03,668 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-08 15:42:03,674 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) near "DATABASE": syntax error
[SQL: CREATE DATABASE CoinIntra]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [12]:
for coin in coins:
    for date in daterange:
        print(f'Processing {date.month_name()} for {coin}...')
        df = getdata(coin, str(date))
        df.to_sql(coin, engine, schema = 'CoinIntra', if_exists='append', index = True)
        sleep(60)
    # Now we have created database COINTEST, a sqllite database, on each iteration a table of each coin in the list is created.
    print(f'finished fetching {coin} data')


Processing September for BTCUSDT...
2022-12-08 15:43:44,358 INFO sqlalchemy.engine.Engine PRAGMA "CoinIntra".table_info("BTCUSDT")
2022-12-08 15:43:44,363 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-08 15:43:44,367 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) unknown database "CoinIntra"
[SQL: PRAGMA "CoinIntra".table_info("BTCUSDT")]
(Background on this error at: https://sqlalche.me/e/14/e3q8)